# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [32]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [33]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/EN_NSYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [34]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)
persistDirectory = './data/processed/faq_data/vectordata/EN_NSYR'
vectorstore = Chroma.from_documents(
    documents=data, embedding=embeddings, persist_directory=persistDirectory)
vectorstore.persist()

Run the cells below to re-use already-generated vector embeddings.

In [1]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/EN_NSYR'
# Retrieve and generate answers using relevant FAQs
vectordb = Chroma(persist_directory=persistDirectory,
                  embedding_function=embeddings)

retriever = vectordb.as_retriever()

In [2]:
from utils import CreateInferenceModifier # Import the function from utils.py
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt_template = """Drawing from your knowledge base, answer the question below.
If you don't know the answer from the provided context, tell me that your training materials don't include this information.
Keep the answer as concise and relevant to the question as possible.

Knowledge Base: {context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template)


# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4000,
             "temperature": 0.2,
             "top_k": 250,
             "top_p": 1,
             "stop_sequences": ["\n\nHuman"],
             }


llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=CreateInferenceModifier("claude", params=modifiers))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [3]:
# Define a function to extract unique URLs used in the retrieved source materials.
from IPython.display import Markdown, display
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string

# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke("What do I need to study at a university in Turkey?")
answer = response["answer"]
prettyResponse = f"## Answer: \n\n {answer} \n\n ## Sources: \n\n {extract_unique_urls(response)}"
Markdown(prettyResponse)

## Answer: 

  Based on the knowledge base provided, to study at a university in Turkey you need:

1) To graduate from high school
2) Meet other conditions for university entrance, such as taking the Foreigner Student Exam (Yabancı Öğrenci Sınavı – YÖS) and obtaining a score that meets the requirements of the university you want to attend
3) If you graduated from a school outside of Turkey or cannot demonstrate previous school attendance, you need to register at Open Education High School (Açık Öğretim Lisesi)
4) Pass the Examination for Foreign Students (YÖS) organized by universities
5) Potentially demonstrate language proficiency depending on university/department requirements
6) Consult university websites/foreign student departments for detailed requirements and announcements

The knowledge base does not provide any additional information beyond these key points regarding what is needed to study at a university in Turkey. Please let me know if you need any clarification or have additional questions. 

 ## Sources: 

 https://multecihaklari.info/services/rights-and-procedures-for-unaccompanied-minors-2/?section=questions&question=16; https://multecihaklari.info/services/rights-and-procedures-for-unaccompanied-minors/?section=questions&question=16; https://multecihaklari.info/services/education/?section=questions&question=9

In [12]:
def query_asylum_system(question):
    from IPython.display import Markdown, display
    # Invoke the LLM chain with the provided question
    response = rag_chain_with_source.invoke(question)

    # Extract the answer from the response
    answer = response["answer"]

    # Format the response using Markdown
    pretty_response = f"## Answer: \n\n{answer}\n\n## Sources:\n\n{extract_unique_urls(response)}"
    return pretty_response

In [13]:
Markdown(query_asylum_system("What steps do I need to take to apply for asylum in Turkey?"))

## Answer: 

 <p>According to the knowledge base provided, to apply for asylum in Turkey:</p>

<ul>
<li>If you are Syrian, you need to apply for "temporary protection" at the Provincial Directorate for Migration Management. It is helpful to provide any Syrian ID documents you have.</li>
<li>If you are from another country, you need to apply for "international protection." You should explain in detail why you cannot return to your country of origin. Providing any ID documents you have and explaining why you fear returning can be helpful.</li> 
<li>You can contact Refugee Rights Turkey for more information on the registration procedures and process after registering.</li>
</ul>

<p>The knowledge base does not include information about applying for asylum from another country or being resettled to another country from Turkey. It focuses on the process for applying for protection within Turkey itself.</p>

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/rights-and-procedures-for-unaccompanied-minors/?section=questions&question=17; https://multecihaklari.info/services/registration-status-2/?section=questions&question=41; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49

In [14]:
Markdown(query_asylum_system(
    "Can you tell me about the rights I have as an asylum seeker in Turkey?"))

## Answer: 

 Based on the knowledge base provided, as an asylum seeker in Turkey you have the following rights:

- The right to legally stay in Turkey while your international protection application is being processed. This protects you from the risk of deportation.

- Access to certain services such as education and healthcare for one year after submitting your international protection application. 

- All services provided by Refugee Rights Turkey to asylum seekers are free of charge. You should report anyone claiming to represent Refugee Rights Turkey who asks for money or other benefits.

The knowledge base does not provide any additional specific information about rights for asylum seekers in Turkey beyond what is summarized here. Please let me know if you need any clarification or have additional questions.

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention/?section=questions&question=30; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention-2/?section=questions&question=30

In [16]:
Markdown(query_asylum_system(
    "How long does the asylum process usually take in Turkey?"))

## Answer: 

 Unfortunately, my training materials do not include specific information about how long the asylum process usually takes in Turkey. The passages state that there is no legal time limit to file an international protection application and advise asylum seekers to approach authorities and initiate asylum proceedings as quickly as possible after arrival. But there is no clear statement about the typical duration of the overall asylum process. Without more context in my knowledge base, I cannot reliably state how long it generally takes. I apologize that I don't have enough information to directly answer the question asked. Please let me know if you need any clarification or have a different question I may be able to assist with.

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=34; https://multecihaklari.info/services/registration-status-2/?section=questions&question=5; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49; https://multecihaklari.info/services/registration-status-2/?section=questions&question=42

In [15]:
Markdown(query_asylum_system(
    "What happens if my asylum application is denied in Turkey?"))

## Answer: 

 Based on the knowledge base provided, if your asylum application is denied in Turkey:

<p>
- The authorities are required to communicate the negative decision and reasons to you or your legal representative. 

- You have the right to appeal the negative decision. If you are in administrative detention, you typically need to appeal to the administrative court within 15 days of being notified of the decision.

- You are allowed to stay in Turkey until there is a final decision on any appeal you file within the time limits. However, you need to notify the Provincial Directorate of Migration Management about your appeal.

- You can access legal support to help with your appeal, such as by calling the Detention Hotline of Refugee Rights Turkey.

So in summary, you have the right to appeal a negative asylum decision in Turkey, and can stay in the country during that appeal process. There are also options to access legal help with filing and pursuing that appeal.

</p>

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=34; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention/?section=questions&question=15; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49

In [17]:
Markdown(query_asylum_system(
    "Can asylum seekers work in Turkey while their application is pending?"))

## Answer: 

 <p>Yes, according to the information provided, asylum seekers can apply for a work permit in Turkey while their application is pending, with some conditions:</p>

<ul>
<li>Asylum seekers can apply for a work permit 6 months after submitting their international protection application.</li>
<li>Work permit requests for asylum seekers are usually submitted by their employer via the e-Government Gateway system.</li>
<li>If the asylum seeker has established their own business in Turkey, they can apply for an "independent work permit" themselves.</li>
</ul>

<p>So in summary, yes asylum seekers can legally work in Turkey while their application is pending, if they meet the requirements around timing and submitting the proper work permit application.</p>

The knowledge base covers the conditions and process for asylum seekers to obtain work permits in Turkey, so I've summarized the key points related to your question in my answer. Let me know if you need any clarification or have additional questions!

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/labor-market-2/?section=questions&question=5; https://multecihaklari.info/services/labor-market-2/?section=questions&question=0; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49

In [18]:
Markdown(query_asylum_system(
    "Where can I get information about the status of my asylum application in Turkey?"))

## Answer: 

 Unfortunately, the knowledge base provided does not include clear information to directly answer the question about where to get information on the status of an asylum application in Turkey. 

The knowledge base talks about what information authorities should provide when an administrative detention order is revoked, including information on one's status in Turkey, rights and entitlements. It also mentions contacting Refugee Rights Turkey or other NGOs to get information about obligations and procedures to follow after release from detention.

However, there is no specific information provided on where to check on the status of an asylum application that has already been submitted. The knowledge base does not seem to cover that particular topic. So I do not have enough context in the provided materials to directly answer the question asked. I apologize that I cannot be more helpful here based on what was provided.

## Sources:

https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention/?section=questions&question=21; https://multecihaklari.info/services/registration-status-2/?section=questions&question=41; https://multecihaklari.info/services/rights-and-procedures-for-persons-in-administrative-detention-2/?section=questions&question=21

In [19]:
Markdown(query_asylum_system(
    "Are there specific documents I need to provide when applying for asylum in Turkey?"))

## Answer: 

 Based on the information provided, if you are a Syrian national applying for temporary protection in Turkey, it is helpful to present your ID documents from Syria when applying. 

If you are applying for international protection from a country other than Syria, while not required, it will be helpful to share with the migration authorities any ID documents you have brought with you. You are also expected to explain in detail the reasons why you cannot go back to your country of origin.

The knowledge base does not specify any other required documents for applying for asylum in Turkey. It focuses more on explaining your situation and why you fear returning to your home country.

## Sources:

https://multecihaklari.info/services/registration-status-2/?section=questions&question=3; https://multecihaklari.info/services/rights-and-procedures-for-unaccompanied-minors/?section=questions&question=17; https://multecihaklari.info/services/registration-status-2/?section=questions&question=49